In [107]:
import pandas as pd
import numpy as np
import os

In [197]:
# # example air quality file
# df_a= pd.read_csv("data/annual_aqi_by_county_1980.csv")
# df_a.head()

In [214]:
# exampel health file
# df_h = pd.read_csv("data/IHME_USA_COUNTY_MORTALITY_RATES_1980_2014_ALABAMA_Y2017M05D19.csv")
# df_h.head()

In [69]:
def rename_county_h(df, state):
# df should have "location_name" as the index
# state is string
# remove " county" from the county name, prepend state name
# apply to each state mortality rate file 
    new_index = []
    for old_name in df.index:
    
        end = len(old_name)
        delete = len(" County")
        new = state+"_"+old_name[0:end-delete]
        new_index.append(new.lower())
        
    df.index = new_index
    return df

In [70]:
#get collection of tables each for one code, one county, one year
# NOT USED

# def transform_state(df, codes):

#     df_by_code = {}
#     #   get rid of state summaries
#     state = df.iloc[0]["location_name"]
#     df_h = df[df["FIPS"] >100]
#     df_h.set_index('location_name',inplace=True)
#     years = df_h['year_id'].drop_duplicates()
    
#     #get tables by year,code
#     for c in codes:
#         df_by_code[c]={}
#         dfc = df_h[df_h["cause_id"] == c]
#         #   organize metadata by county
#         df_meta = dfc[['location_id', 'FIPS', 'cause_id', 'cause_name']].drop_duplicates()
#         for y in years:
#             dfy = dfc[dfc['year_id']==y]
#             df_genders = pd.pivot_table(dfy, values='mx', index='location_name', columns=['sex'])
#             df_genders["Year"] = y
#             df_genders.rename(index=str, columns={"Both": "both_gender_resp_mort", "Female": "female_resp_mort", "Male": "male_resp_mort"}, inplace=True)
#             df_by_county = pd.concat([df_meta, df_genders], axis=1, join='inner')
#             rename_county_h(df_by_county, state)
#             df_by_code[c][y] =df_by_county
#     return df_by_code

In [180]:
#get single table from a health file for one state, one code, one year
# df is dataframe of entire health file for one state
# cause of dealth code
def get_single_table_health(df, code, year):

#     get rid of state summaries
    state = df.iloc[0]["location_name"]
    df_h = df[df["FIPS"] >100]
#     select year
    df_h = df_h[df_h['year_id']==year]
    df_h.set_index('location_name',inplace=True)
    
#     select code
    dfc = df_h[df_h["cause_id"] == code]
    
#     organize metadata by county
    df_meta = dfc[['location_id', 'FIPS', 'cause_id', 'cause_name']].drop_duplicates()
        
#     transform mortality rates by gender
    df_genders = pd.pivot_table(dfc, values='mx', index='location_name', columns=['sex'])
    df_genders.rename(index=str, columns={"Both": "both_gender_resp_mort", "Female": "female_resp_mort", "Male": "male_resp_mort"}, inplace=True)
    df_by_county = pd.concat([df_meta, df_genders], axis=1, join='inner')
    return rename_county_h(df_by_county, state)

In [183]:
# transform the air quality data to be indexed by statename_countyname 
# normalized the air quality metrics by number of days reorded in a location 
def transform_air(df_a):
    # normalize values
    df_a.columns.values
    to_norm = ['Good Days','Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days','Days CO',
       'Days NO2', 'Days Ozone', 'Days SO2', 'Days PM2.5', 'Days PM10']
    
    for c in to_norm:
        df_a[c] = df_a[c]/df_a['Days with AQI']

    df_a["location_name"] = (df_a['State']+"_"+df_a['County']).str.lower()
    df_a.drop(['State','County'], axis=1, inplace=True)
    df_a.set_index('location_name', inplace=True)
    return df_a

In [201]:
directory = 'data'
health_files = []
air_files = []
years = []
joined =pd.DataFrame()

# get health files
for filename in os.listdir(directory):        
    if filename.startswith("IHME"): 
        health_files.append(os.path.join(directory, filename))
        
codes = [508, 515, 519]
for c in codes:
    print("compiling code: " + str(c))
    
    # get health data across all states for one year
    for filename in os.listdir(directory):
        if filename.startswith("annual_aqi"):
#             print(filename)
            year = int(filename[len(filename)-8 :len(filename)-4])
#             print(year)
            air_file = os.path.join(directory,filename)
            dfa = transform_air(pd.read_csv(air_file))
#             print("got transformed air file")
        
#           get first table
            df = pd.read_csv(health_files[0])
            df = get_single_table_health(df, c, year)
            
#             print("health file 0")
#             i=1
            for h in health_files[1:]:
#                 print("health file " + str(i))
                df_h = pd.read_csv(h)
                df = df.append(get_single_table_health(df_h, c, year))
                i+=1
            joined = pd.concat([df, dfa], axis=1, join='inner')
#             print("got joined ")
            print(joined.shape)
            joined.to_csv("data_cleaned/"+str(year)+"_"+str(c)+"_data.csv")
            print("data_cleaned/"+str(year)+"_"+str(c)+"_data.csv")